In [44]:
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from tqdm.notebook import tqdm

In [45]:
class ConvolutionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ConvolutionBlock, self).__init__()

        self.convs = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                      kernel_size=3, padding=0, stride=1),
            nn.BatchNorm2d(out_channels),
            nn.MaxPool2d(2),
            nn.ReLU()
        )

    def forward(self, X):
        return self.convs(X)

In [25]:
class Classifier(nn.Module):
    def __init__(self, in_channels,  num_classes) -> None:
        super(Classifier, self).__init__()

        conv1 = ConvolutionBlock(in_channels=in_channels, out_channels=8)
        conv2 = ConvolutionBlock(in_channels=8, out_channels=16)
        conv3 = ConvolutionBlock(in_channels=16, out_channels=32)

        self.layers = nn.Sequential(conv1, conv2, conv3)

        self.fc = nn.Sequential(
            nn.Linear(in_features=1152, out_features=40),
            nn.ReLU(),
            nn.Linear(in_features=40, out_features=20),
            nn.ReLU(),
            nn.Linear(in_features=20, out_features=num_classes),
            nn.LogSoftmax(dim=-1)
        )

    def forward(self, X):
        X = self.layers(X)
        X = X.view(X.shape[0], -1)
        X = self.fc(X)
        return X

In [40]:
def get_dataloaders(path: str, training: bool):
    transforms = None
    if training:
        transforms = torchvision.transforms.Compose([
            torchvision.transforms.Resize((64, 64)),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.RandomHorizontalFlip()
        ])
    else:
        transforms = torchvision.transforms.Compose([
            torchvision.transforms.Resize((64, 64)),
            torchvision.transforms.ToTensor()
        ])

    data = torchvision.datasets.ImageFolder(
        path, transform=transforms)

    data_loader = DataLoader(data, batch_size=32, shuffle=training)

    return data_loader

In [46]:
def train_model(model=Classifier(in_channels=3, num_classes=4)):

    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
    EPOCHS = 30
    train_loader = get_dataloaders(path='Mask_Data/train', training=True)
    valid_loader = get_dataloaders(path='Mask_Data/val', training=False)
    model.train()
    for epoch in tqdm(range(EPOCHS)):
        # for batch_idx, (inputs, labels) in enumerate(train_loader):
        for batch in train_loader:
            inputs, labels = batch

            # Forward pass
            y_pred = model.forward(inputs)
           # print('y_pred shape: ', y_pred.shape)

            # Calculating loss
            loss = criterion(input=y_pred, target=labels)

            # BackPropagation
            loss.backward()
            optimizer.step()

            # Zero Gradients
            optimizer.zero_grad()

        train_loss.append(loss)

        with torch.no_grad():
            for batch in valid_loader:
                val_input, val_labels = batch
                preds = model.forward(val_input)

                val_loss = criterion(input=preds, target=val_labels)

        print('Epoch: {}, Loss: {}'.format(epoch, loss.item()))
    return model


model = train_model()

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 0, Loss: 0.2563569247722626
Epoch: 1, Loss: 0.3352801501750946
Epoch: 2, Loss: 0.11568420380353928
Epoch: 3, Loss: 0.15402397513389587
Epoch: 4, Loss: 0.019899113103747368
Epoch: 5, Loss: 0.39992621541023254
Epoch: 6, Loss: 0.00027735871844924986
Epoch: 7, Loss: 0.029714811593294144
Epoch: 8, Loss: 0.01656620390713215
Epoch: 9, Loss: 0.03753550723195076
Epoch: 10, Loss: 0.0061338203959167
Epoch: 11, Loss: 0.0008362559601664543
Epoch: 12, Loss: 0.0003340213734190911
Epoch: 13, Loss: 0.0010011944686993957
Epoch: 14, Loss: 0.05055258050560951
Epoch: 15, Loss: 0.024166906252503395
Epoch: 16, Loss: 0.0023738828022032976
Epoch: 17, Loss: 0.00425967900082469
Epoch: 18, Loss: 0.0017131709028035402
Epoch: 19, Loss: 0.05694277584552765
Epoch: 20, Loss: 0.013967734761536121
Epoch: 21, Loss: 0.0017920159734785557
Epoch: 22, Loss: 0.0003140699991490692
Epoch: 23, Loss: 0.19382494688034058
Epoch: 24, Loss: 4.0689569686946925e-06
Epoch: 25, Loss: 0.020323496311903
Epoch: 26, Loss: 0.0017619834